In [ ]:
# 각 코드 셀의 실행시간 측정 모듈
!pip install ipython-autotime

%load_ext autotime

* M4 dataset download and extract

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/m4.zip" -d /content/dataset

Read dataset -> dataframe

In [ ]:
import os
import pandas as pd
import numpy as np
from google.colab import files
import tqdm
from collections import OrderedDict
from glob import glob
from shutil import copyfile

time: 344 ms (started: 2022-03-15 06:18:03 +00:00)


In [ ]:
DATASET_PATH = r"/content/dataset/m4/"

m4_info = pd.read_csv("/content/dataset/m4/m4_info.csv") 
m4_ids = m4_info.M4id.values

time: 162 ms (started: 2022-03-15 06:18:03 +00:00)


In [ ]:
def build_cache(files: str, cache_path: str) -> None:
    timeseries_dict = OrderedDict(list(zip(m4_ids, [[]] * len(m4_ids))))

    for train_csv in glob(os.path.join(DATASET_PATH, files)):
        print(train_csv)
        dataset = pd.read_csv(train_csv)
        dataset.set_index(dataset.columns[0], inplace=True)
        for m4id, row in dataset.iterrows():
            values = row.values
            timeseries_dict[m4id] = values[~np.isnan(values)]
    np.array(list(timeseries_dict.values())).dump(cache_path)

time: 12.2 ms (started: 2022-03-15 06:18:03 +00:00)


In [ ]:
if os.path.isfile("/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/training.npz"):
    !cp "/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/training.npz" "/content/dataset/m4/training.npz"
else:
    build_cache('*-train.csv', os.path.join(DATASET_PATH, 'training.npz'))
    !cp "/content/dataset/m4/training.npz" "/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/training.npz"    

time: 5.69 s (started: 2022-03-15 06:18:03 +00:00)


In [ ]:
if os.path.isfile("/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/test.npz"):
    !cp "/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/test.npz" "/content/dataset/m4/test.npz"
else:
    build_cache('*-test.csv', os.path.join(DATASET_PATH, 'test.npz'))
    !cp "/content/dataset/m4/test.npz" "/content/drive/MyDrive/Colab Notebooks/N-BEATS/dataset/test.npz"    

time: 1.67 s (started: 2022-03-15 06:18:09 +00:00)


In [ ]:
class M4Dataset:
    ids: np.ndarray
    groups: np.ndarray
    frequencies: np.ndarray
    horizons: np.ndarray
    values: np.ndarray


def load(fp: str) -> 'M4Dataset':
    dataset = M4Dataset()
    dataset.ids = m4_info.M4id.values
    dataset.groups=m4_info.SP.values
    dataset.frequencies=m4_info.Frequency.values
    dataset.horizons=m4_info.Horizon.values
    dataset.values=np.load(fp, allow_pickle=True)

    return dataset

dataset = load(os.path.join(DATASET_PATH, 'training.npz'))

time: 3.49 s (started: 2022-03-15 06:18:13 +00:00)


In [ ]:
class M4Meta:
    seasonal_patterns = ['Yearly', 'Quarterly', 'Monthly', 'Weekly', 'Daily', 'Hourly']
    horizons = [6, 8, 18, 13, 14, 48]
    frequencies = [1, 4, 12, 1, 1, 24]
    horizons_map = {
        'Yearly': 6,
        'Quarterly': 8,
        'Monthly': 18,
        'Weekly': 13,
        'Daily': 14,
        'Hourly': 48
    }
    frequency_map = {
        'Yearly': 1,
        'Quarterly': 4,
        'Monthly': 12,
        'Weekly': 1,
        'Daily': 1,
        'Hourly': 24
    }

time: 5.7 ms (started: 2022-03-15 06:18:31 +00:00)


* Select#1 - Para. of Paper

In [ ]:
#generic
history_size = {
    'Yearly': 1.5,
    'Quarterly': 1.5,
    'Monthly': 1.5,
    'Weekly': 10,
    'Daily': 10,
    'Hourly': 10
}

iterations = {
    'Yearly': 15000,
    'Quarterly': 15000,
    'Monthly': 15000,
    'Weekly': 5000,
    'Daily': 5000,
    'Hourly': 5000
}

layer_size = 512
layers = 4
stacks = 30

batch_size = 1024
learning_rate = 0.001

logging_frequency = 100
snapshot_frequency = 1000

# Ensemble parameters
repeats = 10
lookbacks = [2, 3, 4, 5, 6, 7]
losses = ['MASE', 'MAPE', 'SMAPE']

* Select#1-2 - Para. of Test

In [ ]:
#generic
history_size = {
    'Yearly': 1.5,
    'Quarterly': 1.5,
    'Monthly': 1.5,
    'Weekly': 10,
    'Daily': 10,
    'Hourly': 10
}

iterations = {
    'Yearly': 15,
    'Quarterly': 15,
    'Monthly': 15,
    'Weekly': 5,
    'Daily': 5,
    'Hourly': 5
}

layer_size = 512
layers = 4
stacks = 30

batch_size = 1024
learning_rate = 0.001

logging_frequency = 100
snapshot_frequency = 1000

# Ensemble parameters
repeats = 2
lookbacks = [2, 3, 4, 5, 6, 7]
losses = ['MASE', 'MAPE', 'SMAPE']

time: 12.5 ms (started: 2022-03-15 06:20:52 +00:00)


In [ ]:
def group_values(values: np.ndarray, groups: np.ndarray, group_name: str) -> np.ndarray:
    """
    Filter values array by group indices and clean it from NaNs.

    :param values: Values to filter.
    :param groups: Timeseries groups.
    :param group_name: Group name to filter by.
    :return: Filtered and cleaned timeseries.
    """
    return np.array([v[~np.isnan(v)] for v in values[groups == group_name]])

time: 3.04 ms (started: 2022-03-15 06:20:58 +00:00)


In [ ]:
lookback = lookbacks[1] # 3

In [ ]:
forecasts = []
for seasonal_pattern in M4Meta.seasonal_patterns:
    history_size_in_horizons = history_size[seasonal_pattern]
    horizon = M4Meta.horizons_map[seasonal_pattern]
    input_size = lookback * horizon

    # Training Set
    training_values = group_values(dataset.values, dataset.groups, seasonal_pattern)

    training_set = TimeseriesSampler(timeseries=training_values,
                                    insample_size=input_size,
                                    outsample_size=horizon,
                                    window_sampling_limit=int(history_size_in_horizons * horizon))

    if model_type == 'interpretable':
        model = interpretable(input_size=input_size, output_size=horizon)
    elif model_type == 'generic':
        model = generic(input_size=input_size, output_size=horizon)
    else:
        raise Exception(f'Unknown model type {model_type}')

    # Train model
    snapshot_manager = SnapshotManager(snapshot_dir=os.path.join(self.root, 'snapshots', seasonal_pattern),
                                        total_iterations=iterations[seasonal_pattern])
    model = trainer(snapshot_manager=snapshot_manager,
                    model=model,
                    training_set=iter(training_set),
                    timeseries_frequency=M4Meta.frequency_map[seasonal_pattern],
                    loss_name=loss,
                    iterations=iterations[seasonal_pattern])

    # Build forecasts
    x, x_mask = map(to_tensor, training_set.last_insample_window())
    model.eval()
    with t.no_grad():
        forecasts.extend(model(x, x_mask).cpu().detach().numpy())

In [ ]:
        forecasts_df = pd.DataFrame(forecasts, columns=[f'V{i + 1}' for i in range(np.max(M4Meta.horizons))])
        forecasts_df.index = dataset.ids
        forecasts_df.index.name = 'id'
        forecasts_df.to_csv(os.path.join("/content/", 'forecast.csv'))